In [1]:
!pip install -q -U git+https://github.com/mljar/mljar-supervised.git@master

In [11]:
## 데이터 로드 
import pandas as pd

train = pd.read_csv("../input/minyoung-kim/fanal220808.csv")
train.head()

In [12]:
train.info()

In [13]:
from supervised.preprocessing.eda import EDA

train = train
X = train[train.columns[:-1]]
y = train["frequency"]

EDA.extensive_eda(X, y, save_path = './')

In [18]:
#Explain
from supervised import AutoML

explain = AutoML(total_time_limit=60*20,
                model_time_limit = 120,
                mode = "Explain",
                eval_metric="rmse",
                algorithms = ['Xgboost','Decision Tree', 'Linear', 'Random Forest', 
                              'Extra Trees', 'LightGBM', 'CatBoost', 'Neural Network', 'Nearest Neighbors'],
                ml_task = 'regression',
                train_ensemble=True,
                n_jobs = -1,
                random_state = 2022)
explain.fit(X,y)

In [19]:
pd.set_option('display.max_rows', None)
explain.get_leaderboard()

Catboost,
Ensemble 

In [ ]:
explain.report()

In [21]:
#"Perform"

perform = AutoML(total_time_limit=60*10,
                model_time_limit = 60,
                mode = "Perform",
                eval_metric="rmse",
                algorithms = ['Xgboost','Decision Tree','Linear', 'Random Forest', 'Extra Trees', 
                              'LightGBM', 'CatBoost', 'Neural Network', 'Nearest Neighbors'],
                ml_task = 'regression',
                train_ensemble=True,
                n_jobs = -1,
                validation_strategy = { 
                    "validation_type": "kfold",
                    "k_folds": 5,
                    "shuffle": True,
                    "stratify": True,
                    "random_seed": 46
                },
                random_state = 2022)
perform.fit(X,y)

In [22]:
pd.set_option('display.max_rows', None)
perform.get_leaderboard()
perform.report()

In [25]:
#Compete
compete = AutoML(total_time_limit=60*10,
                model_time_limit = 60,
                mode = "Compete",
                eval_metric="rmse",
                algorithms =  ['Linear', 'Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 
                               'Xgboost', 'CatBoost', 'Neural Network' ,'Nearest Neighbors'],
                ml_task = 'regression',
                train_ensemble=True,
                n_jobs = -1,
                random_state = 2022)
compete.fit(X,y)

In [26]:
pd.set_option('display.max_rows', None)
compete.get_leaderboard()
compete.report()

In [ ]:
#Optuna

AutoML(mode="Optuna", optuna_time_budget=3600)
compete = AutoML(total_time_limit=60*10,
                model_time_limit = 60,
                mode = "Compete",
                eval_metric="rmse",
                algorithms = ['Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost','CatBoost']
                ml_task = 'regression',
                train_ensemble=True,
                n_jobs = -1,
                validation_strategy = { 
                    "validation_type": "kfold",
                    "k_folds": 5,
                    "shuffle": True,
                    "stratify": True,
                    "random_seed": 46
                }
                random_state = 2022)
compete.fit(X,y)

# 예측

In [27]:
vali = pd.read_csv("../input/minyoungkim/vali.csv")
vali = vali.drop("frequency",axis = 1)
vali.head()

In [31]:
sub = pd.read_csv("../input/minyoungkim/sub.csv",encoding= 'euc-kr')
sub.head()

In [32]:
pred = compete.predict(vali)

In [33]:
sub.loc[:,"frequency"] 

In [35]:
sub.loc[:,"frequency"] =pred

In [36]:
sub.head()

In [37]:
sub.to_csv("sub_compete.csv",index=False)